# `.groupby()` and `.agg()` - Part 2

The purpose of this tutorial is to demonstrate how to use `.groupby()` and `.agg()` with user defined functions.

In service of this objective, our analysis goal will be to calculate three monthly statistics for several ETFs during the year of 2020.  The three statistics we will calculate are:

1. average daily volume

2. monthly return

3. monthly volatility

### Loading Packages

Let's load the packages that we will need.

In [ ]:
##> import numpy as np
##> import pandas as pd
##> import pandas_datareader as pdr




### Reading-In Data

In this tutorial we will be working with price data for the Select Sector SPDR ETFs.  Each of these funds tracks a particular subset (sector) of the SP&500 Index.  For example, XLF tracks the financial sector and has major holdings in JP Morgan, Wells Fargo, and Bank of America.

Let grab the data from Yahoo:

In [ ]:
##> pd.options.display.max_rows = 25
##> lst_symbols = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU',]
##> df_etf = pdr.get_data_yahoo(lst_symbols, start='2020-01-01', end='2020-12-31')
##> df_etf = df_etf.round(2)
##> df_etf.head()




This data is not as tidy as we would like.  Let's use method chaining to perform a series of data munging operations.

In [ ]:
##> df_etf = \
##>     (
##>     df_etf
##>         .stack(level='Symbols') #pivot the table
##>         .reset_index() #turn date into a column 
##>         .sort_values(by=['Symbols', 'Date']) #sort
##>         .rename(columns={'Date':'date', 'Symbols':'symbol', 'Adj Close':'adj_close','Close':'close', 
##>                          'High':'high', 'Low':'low', 'Open':'open', 'Volume':'volume'}) #renaming columns
##>         [['date', 'symbol','open', 'high', 'low', 'close', 'volume', 'adj_close']] #reordering columns
##>     )
##> df_etf




**Coding Challenge:** Use a `DataFrame` attribute to determine the number of rows and columns in `df_etf`.

### Exploring and Cleaning the Data

As we can see from the coding challenge, this data set is large (by our standards).  Whenever I encounter a new data set that I can't look at in its entirety, I like to do a bit of exploration via the built-in `pandas` methods.

We know we have a variety of ETFs in our data, but it would be useful to know how many (especially if we were expecting a certain number).

In [ ]:
##> print(df_etf['symbol'].unique())
##> print(df_etf['symbol'].unique().size)




**Coding Challenge:** What `DataFrame` attribute could we use to check the data types of the columns of `df_etf`?

When I work with a time series of daily prices, I like to check the first and last trade dates that are represented in the data.

In [ ]:
##> print(df_etf['date'].min())
##> print(df_etf['date'].max())




Here is what we know about our data set thus far:

1. 10 different ETFs are represented.

1. Prices are coming from the entirety of 2020.


Here are some things that we aren't necessarily sure of that would be worth checking in a high-stakes situation:

1. Is there a row/price for each symbol on each trade date?

1. Is there ever more than one row/price for a given symbol on a given trade date?

We won't bother answering these questions for the purposes of this tutorial, but these are the types of data-integrity questions I will often try to answer when encountering a new data set.

### Calculating Daily Returns with `groupby()`

Our ultimate goal is to calculate monthly returns and monthly volatilities for each ETF in `df_etf`.  These quantities are both functions of daily returns.  So, our first order of business is to calculate daily returns. 

In a previous tutorial we calculated daily returns in a simple vectorized fashion.  Unfortunately, we can't use the exact same approach here because there are multiple ETFs in the data set.

To overcome this challenge we will use our first application of `.groupby()`.

Here is the `.groupby()` code that calculates daily returns for each ETF.

In [ ]:
##> # sorting values to get everything in the right order
##> df_etf.sort_values(['symbol', 'date'], inplace=True)
##> 
##> # vectorized return calculation
##> df_etf['dly_ret'] = \
##>     df_etf['close'].groupby(df_etf['symbol']).pct_change()
##> df_etf.head()




### Adding `year` and `month` Columns

The ultimate goal is to calculate monthly statistics for each of the ETFs in our data set.

As a preliminary step, let's add a month and year column to the `df_etf` by utilizing the `.dt` attribute that `pandas` provides for date columns.

In [ ]:
##> df_etf['year'] = df_etf['date'].dt.year
##> df_etf['month'] = df_etf['date'].dt.month
##> df_etf[['date', 'year', 'month']].head()




Let's do a quick data-integrity check:  There are 10 ETFs in our data set and there are 12 months in a year, so the number of symbol-year-month combinations should be 120.

The following code counts the number of rows associated with each symbol-year-month combination and puts that data into a `DataFrame`.

In [ ]:
##> df_num_rows = \
##>     df_etf.groupby(['symbol', 'year', 'month']).size().reset_index()
##> df_num_rows.head()




**Coding Challenge:** Confirm that there are the correct number of symbol-year-month combinations in `df_num_rows`.

Now that we've added the `year` and `month` columns we can proceed to calculating our monthly statistics.

### Average Daily Volume

Let's start with the most straight-forward calculation: average daily volume, over each month, for all 10 of the ETFs in our data set.  

This amounts to:

1. grouping by `symbol`, `month`, and `year`

1. applying the built-in `np.mean()` function to the `volume` column 

In [ ]:
##> df_volume = \
##>     df_etf.groupby(['symbol', 'year', 'month'])['volume'].agg([np.mean]).reset_index()
##> df_volume.rename(columns={'mean':'avg_volume'}, inplace=True)
##> df_volume.head()




**Coding Challenge:** Calculate the maximum daily volume for each symbol, *over the entire year*.

### Monthly Returns

Next, let's calculate monthly returns for each of the ETFs in our data set.  This amounts to:

1. grouping by `symbol`, `month`, and `year`

1. applying an aggregation function to the `daily_returns` column

These are the same two steps that we have done in our previous aggregation examples.  However, there is one additional wrinkle that we are going to have to contend with. 


In the previous section, we used simple built-in aggregation funtions available through `numpy`, such as `np.max` and `np.mean`.  Calculating monthly returns from daily returns is a little more complicated.

Thus, we are going to have to first create a custom function for calculating monthly returns from daily returns, and then use this custom function in `.agg()`.

The following code defines our monthly returns function in terms of daily returns:

In [ ]:
##> def monthly_ret(dly_ret):
##>     return np.prod(1 + dly_ret) - 1




Now we can apply `monthly_ret` for all of our ETFs using the following code.

In [ ]:
##> df_ret = \
##>     df_etf.groupby(['symbol', 'month', 'year'])['dly_ret'].agg([monthly_ret]).reset_index()
##> df_ret.head()




We can see from our calculation that in March of 2020 XLB had a monthly return of -14.6%.

**Coding Challenge:** Which ETF had the highest single monthly return in all of 2020?  What was the month?

### Monthly Volatility

Let's use a similar process to calculate the monthly volatility for each of the ETFs.

We begin by defining a custom function that calculates the monthly volatility from daily returns.  Recall that industry convention is to state these volatilities in annualized terms.

In [ ]:
##> def monthly_vol(dly_ret):
##>     return np.std(dly_ret) * np.sqrt(252)




We can now use our `monthly_vol` function in to perform an aggregating calculation.

In [ ]:
##> df_vol = \
##>     df_etf.groupby(['symbol', 'month', 'year'])['dly_ret'].agg([monthly_vol]).reset_index()
##> df_vol.head()




**Coding Challenge:**  What was the volatility for XLF in December 2018?

### Combining Metrics - `inner join`

Now, suppose that we want to combine our three metrics into one report - meaning that we want them organized into one `DataFrame` in an easy to read fashion.

One way to do this is to use the `pandas.merge()` method that we learned in the previous tutorial to join together `df_volume` (average daily volume), `df_ret` (monthly returns), and `df_vol` (monthly volatility). 

In [ ]:
##> df_joined = \
##>     (
##>     df_volume
##>         .merge(df_ret, on=['symbol', 'year', 'month'])
##>         .merge(df_vol, on=['symbol', 'year', 'month'])
##>     )
##> df_joined.head()




## Combining Metrics - multiple aggregation

Another way to combine all our statistics into a single `DataFrame` is to supply all of our custom aggregation fuctions as arguments to the `.agg()` function in one shot.  

Here is what that looks like:

In [ ]:
##> # defining aggregations
##> agg_funcs = \
##>     {'volume':[np.mean], 'dly_ret':[monthly_ret, monthly_vol]}
##> 
##> # performing all aggregations all three aggregations at once
##> df_joined = \
##>     df_etf.groupby(['symbol', 'month', 'year']).agg(agg_funcs).reset_index()
##> 
##> # looking at the data frame
##> df_joined.head()




Notice that the input into the `.agg()` method is a `dict` whose elements are pairs that look like: 

`'column_name':[list_of_aggregating_functions`].

**Code Challenge:** Modify the code above to add maximum daily volume to the report.

## Related Reading

*PDSH* - 3.7 - Combining Datasets: Merging and Joining

*PDSH* - 3.8 - Aggregation and Grouping

*Python for Data Analysis (McKinney)* - Chapter 9 (pp 251-274) Data Aggregation and Grouping Operations